In [1]:
from typing import Dict, Optional
from datasets import Dataset, load_dataset, concatenate_datasets
import os
import glob
import json
import copy
import threading
import time
import random
from typing import Dict, Optional
import requests
import numpy as np
import re

import fasttext
# lang_detect = fasttext.load_model('../fastchat/modules/fasttext/lid.176.bin')

from fastchat.modules.answer_refiner import generate_refiner

import chromadb
from chromadb.config import Settings
import random
from fastchat.modules.embedder_adapter import Embedder, get_embedder
from fastchat.conversation import (
    SeparatorStyle,
)
from fastchat.model.model_adapter import get_conversation_template
import copy
from fastchat.train.data_modules.sft_dataset import load_sft_dataset, combine_dataset
from fastchat.train.data_modules.dpo_dataset import load_dpo_dataset
from fastchat.train.data_modules.dedup import (
    dedup_by_similarity,
    dedup_non_pair,
    dedup_repetition,
    dedup_math,
    dedup_too_much_token,
    dedup_short,
)
def random_select(dataset, num_select):
    return dataset.select(np.random.choice(list(range(len(dataset))), num_select))

/home/ados/anaconda3/envs/fastchat_2023dec/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# def dedup_by_similarity(dataset, prompt_template='chat-orca', target_text_len=100, n_results=100, distance_threshold = 0.6):
_dataset = train_dataset
prompt_template='vicuna'
target_text_len=100
n_results=100
distance_threshold = 0.35
    
if prompt_template == 'chat-orca':
    conv = get_conversation_template(prompt_template)
    system_message = conv.system_message
    sep_style = conv.sep_style
    sep = conv.sep
    prompt_user, prompt_bot = conv.roles

    len_sep_style = 0
    if sep_style == SeparatorStyle.ADD_COLON_TWO:
        len_sep_style = 1

    len_front = len(system_message) + len(sep) + len(prompt_user) + len_sep_style + 1
    len_rear = len(sep) + len(prompt_bot) + len_sep_style
    def filter_question(data):
        return { 
            # **data,
            'prompt': data['prompt'][len_front:-len_rear][:target_text_len]
        }

if prompt_template == 'vicuna':
    def filter_question(data):
        return {
            'prompt': data['conversations'][0]['value'][:target_text_len]
        }

question_dataset = _dataset.map(filter_question)

chroma_client = chromadb.Client(Settings(anonymized_telemetry=False))
embedder = get_embedder("ddobokki/klue-roberta-base-nli-sts-ko-en")
collection = chroma_client.create_collection(name="context", embedding_function=embedder.embed, metadata={"hnsw:space": "cosine"})
ids = []
# add
texts = question_dataset['prompt']
last_id = -1
new_ids = [f"id{i+last_id+1}" for i in range(len(texts))]
ids += new_ids
collection.add(documents=texts, ids=new_ids)

query_ids = copy.deepcopy(new_ids)
selected_ids = []
duplicated_ids = []

weird_ids = []
error_ids = []
while query_ids:
    current_id = random.choice(query_ids)
    if current_id in selected_ids:
        print("Warning: this is weird..")
        weird_ids.append(current_id)
        continue
    selected_ids.append(current_id)
    search_strings = [texts[int(current_id[2:])]]
    if collection.count() == 0:
        print("Warning: collection is empty. Forced break")
        break
    result = collection.query(query_texts=search_strings, n_results=min(n_results, len(query_ids)), include=['distances']) #'documents'

    search_ids = result['ids'][0]
    distances = result['distances'][0]
    remove_ids = []
    for idx in range(len(search_ids)):
        sid = search_ids[idx]
        dist = distances[idx]
        if dist < distance_threshold:
            remove_ids.append(sid)

    for rid in remove_ids:
        if rid in query_ids:
            query_ids.remove(rid)
            
    if remove_ids:
        duplicated_ids += remove_ids
        collection.delete(ids=remove_ids)
    else:
        print("Warning: this is error..")
        error_ids.append(current_id)

    print(f"Total:{len(new_ids)} Selected:{len(selected_ids)} current_dup:{len(remove_ids)} vector_store:{collection.count()} remained:{len(query_ids)} total_dup:{len(duplicated_ids)}", '\t\t\t\t\t', end='\r')

print('finished dedup data:', f"Total:{len(new_ids)} Selected:{len(selected_ids)} current_dup:{len(remove_ids)} vector_store:{collection.count()} remained:{len(query_ids)} total_dup:{len(duplicated_ids)}")

selected_ids = [int(sid[2:]) for sid in set(selected_ids)]

_dataset = _dataset.select(selected_ids)

# return dataset, selected_ids, query_ids

In [189]:
dataset = load_dpo_dataset(dpo_list5[2])

In [ ]:
dataset[2000]

In [190]:
mdpo.append(random_select(dataset, 1000))

In [ ]:
new_dataset_list = []
for d in dataset_list:
    new_dataset_list.append("\"" + d + "\"")
print(' '.join(new_dataset_list))

In [184]:
mdpo = []

In [133]:
msft = []

In [158]:
dataset_path = dataset_list[14]
print(dataset_path)
dataset = load_sft_dataset(dataset_path)
dataset

/data/llm_datasets/custom/deduped2/sharegpt_V3_format_translation(koen)-10000.json


Dataset({
    features: ['id', 'conversations', 'task'],
    num_rows: 10000
})

In [159]:
msft.append(random_select(dataset, 50))

In [191]:
combined_dataset = concatenate_datasets(mdpo)

In [161]:
combined_dataset = combined_dataset.shuffle(42)

In [ ]:
combined_dataset[2]

In [ ]:
# from fastchat.train.data_modules.sft_dataset import load_sft_dataset, combine_dataset
combined_dataset = concatenate_datasets([load_sft_dataset(dataset_path) for dataset_path in dataset_list])
combined_dataset.features

In [192]:
new_dataset = []
for data in combined_dataset:
    new_dataset.append(data)    


In [193]:
# combined_dataset.to_json("/data/llm_datasets/custom/ados_sft_v4.json")
with open("/data/llm_datasets/custom/ados/dpo/ados_mdpo_v2.json", "w") as json_file:
    json.dump(new_dataset, json_file)

In [194]:
dpo_dataset = ados_DPODataset("/data/llm_datasets/custom/ados/dpo/ados_mdpo_v2.json")

In [196]:
dpo_datamodule = dpo_dataset.make_dpo_data_module()

Extracting data files: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 1349.52it/s]
Generating train split: 11000 examples [00:01, 8384.35 examples/s]
Map: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 11000/11000 [00:01<00:00, 9377.51 examples/s]


In [ ]:
from collections import defaultdict

"""
get and save language distributions
"""
def get_lang_distribution(dataset_list):
    lang_distribution = {}
    global_lang_dict = {'train': defaultdict(int), 'test': defaultdict(int)}
    for dataset_path in dataset_list:
        # dataset = load_sft_dataset(dataset_path, split=None)
        dataset = load_dpo_dataset(dataset_path, split=None)

        lang_splits = {}
        for split in list(dataset.keys()):
            print(f"{dataset_path}:{split}")
            _dataset = dataset[split]

            lang_dict = defaultdict(int)
            for data in _dataset:
                # conversations = data['conversations']
                conversations = data['chosen_response']
                langs = {}
                len_conv = 0
#                 for conv in conversations:
#                     # _from = conv['from']
#                     # _value = conv['value']
#                     _from = conv['role']
#                     _value = conv['content']

#                     len_conv += len(_value)
#                     lang, conf = lang_detect.predict(_value.replace('\n', ' '))
#                     lang = lang[0]
#                     if lang not in langs:
#                         langs[lang] = 1
#                     else:
#                         langs[lang] += 1
                _value = conversations

                len_conv += len(_value)
                lang, conf = lang_detect.predict(_value.replace('\n', ' '))
                lang = lang[0]
                if lang not in langs:
                    langs[lang] = 1
                else:
                    langs[lang] += 1

#                 if '__label__en' in langs:
#                     langs['__label__en'] -= 1

                if len(langs) == 0:
                    dominent_lang = "empty"
                else:
                    dominent_lang = max(langs)
                
                # if dominent_lang not in lang_dict:
                #     lang_dict[dominent_lang] = 1
                # else:
                global_lang_dict[split][dominent_lang] += 1
                lang_dict[dominent_lang] += 1
            lang_splits[split] = lang_dict
        lang_distribution[dataset_path] = lang_splits
    
    lang_distribution['total'] = global_lang_dict
    
    stat_dict = {'train': {}, 'test': {}}
    for split in ['train', 'test']:
        total_cnt = sum([value for value in global_lang_dict[split].values()])
        for key, value in global_lang_dict[split].items():
            stat_dict[split][key] = f"{value / total_cnt:.2%}"
    
    lang_distribution['stat'] = stat_dict
    
    return lang_distribution

lang_distribution = get_lang_distribution([dpo_list[2]])
lang_distribution

In [6]:
with open("/workspaces/data/llm_datasets/custom/lang_distribution_SFT_v4.json", "w") as json_file:
    json.dump(lang_distribution, json_file, indent=4)

In [ ]:
dataset = load_dataset("json", data_files="/workspaces/data/llm_datasets/custom/deduped/translated_sharegpt_V3_format_ko.json")
dataset

In [ ]:
dataset_koen = dataset['train'].select(range(10000, 15000))
dataset_enko = dataset['train'].select(range(15000, 20000))

In [ ]:
"""
SFT raw data and change format
"""

In [ ]:
new_dataset = []
for data in dataset_enko:
    conversations_ko = data['conversations']
    _id = data['id']
    
    conversations_en = lang_dict['__label__en'][_id]['conversations']
    
    for _idx in range(len(conversations_ko)):
        value_ko = conversations_ko[_idx]['value']
        value_en = conversations_en[_idx]['value']
        if len(value_ko) > 150: # 너무 짧은 데이터는 품질이 좋지 않아 제거
            new_dataset.append({
                'id': f"sharegpt_V3_format_{_id}_{_idx}",
                'task': 'enkotranslation',
                'conversations': [
                                    {'from': 'human', 'value': value_en},
                                    {'from': 'gpt', 'value': value_ko},
                                 ],
            })

In [ ]:
"""make lang_dict(sft)"""
dataset = load_dataset("json", data_files="/data/llm_datasets/custom/kodpo/untranslated/ultrafeedback_binarized.json")
enko_dataset = []
remained_dataset = []

lang_dict = {}
for data in dataset['train']:
    conversations = data['conversations']
    langs = {}
    len_conv = 0
    for conv in conversations:
        _from = conv['from']
        _value = conv['value']

        len_conv += len(_value)
        lang, conf = lang_detect.predict(_value.replace('\n', ' '))
        lang = lang[0]
        if lang not in langs:
            langs[lang] = 1
        else:
            langs[lang] += 1

    if '__label__en' in langs:
        langs['__label__en'] -= 1

    dominent_lang = max(langs)
    if dominent_lang not in lang_dict:
        lang_dict[dominent_lang] = [data]
    else:
        lang_dict[dominent_lang].append(data)
    # break

In [16]:
"""make lang_dict(dpo)"""
dataset = load_dataset("json", data_files="/data/llm_datasets/custom/kodpo/untranslated/truthy-dpo-v0.1.json")
enko_dataset = []
remained_dataset = []

lang_dict = {}
for data in dataset['train']:
    chosen = data['chosen']
    rejected = data['rejected']
    
    langs = {}

    lang_chosen = lang_detect.predict(chosen.replace('\n', ' '))[0][0]
    if lang_chosen not in langs:
        langs[lang_chosen] = 1
    else:
        langs[lang_chosen] += 1
        
    lang_rejected = lang_detect.predict(rejected.replace('\n', ' '))[0][0]
    if lang_rejected not in langs:
        langs[lang_rejected] = 1
    else:
        langs[lang_rejected] += 1

    if '__label__en' in langs:
        langs['__label__en'] -= 1

    dominent_lang = max(langs)
    if dominent_lang not in lang_dict:
        lang_dict[dominent_lang] = [data]
    else:
        lang_dict[dominent_lang].append(data)
    # break

Extracting data files: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 902.97it/s]
Generating train split: 1016 examples [00:00, 21155.38 examples/s]


In [17]:
for key, value in lang_dict.items():
    print(key, len(value))

__label__en 1016


In [ ]:
import requests
api_server_url = "http://localhost:41002"
def count_total_token(conversations):
    num_token = 0
    for conv in conversations:
        input_json = {
            "model_name": "MingAI-70B-chat-orca_v0.42_2_dpo-GPTQ",
            "prompt": conv['value'],
        }

        ret = requests.post(api_server_url + "/count_token", json=input_json)

        output_json = ret.json()
        num_token += output_json['count']
    return num_token


for _idx in range(10):
    num_total_token = count_total_token(lang_dict['__label__en'][_idx])
    print(num_total_token)

In [ ]:
new_dataset += lang_dict['__label__pt'] + lang_dict['__label__es'] + lang_dict['__label__de'] + lang_dict['__label__zh'] + lang_dict['__label__fr']

In [ ]:
with open("/data/llm_datasets/custom/vicuna_format/lima_vicuna_format_ko.json", "w") as json_file:
    json.dump(new_dataset, json_file)

In [ ]:
new_dataset[2]

In [118]:
id_row = 'Ko_distilabel-math-preference-dpo_1'
filtered_row = dataset.filter(lambda sample: sample['id'] == id_row)
filtered_row[0]

Filter: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2418/2418 [00:00<00:00, 111092.18 examples/s]


{'input': 'What is the result of substituting `x = 5` and `y = -2` into the expression `2x^2 - 3y + 7`?.\n Take a deep breath, think step by step, and give an accurate response',
 'chosen': "To find the result of substituting `x = 5` and `y = -2` into the expression `2x^2 - 3y + 7`, we will substitute these values for the variables `x` and `y` in the given expression and simplify.\n\nSubstituting `x = 5` and `y = -2` into the expression, we get:\n\n2(5)^2 - 3(-2) + 7\n\nSimplifying, we have:\n\n2(25) + 6 + 7\n\nNow, let's perform the operations:\n\n50 + 6 + 7 = 63\n\nTherefore, the result of substituting `x = 5` and `y = -2` into the expression `2x^2 - 3y + 7` is 63.",
 'task': 'dpo',
 'id': 'Ko_distilabel-math-preference-dpo_1',
 'rejected': 'To find the result of substituting `x = 5` and `y = -2` into the expression `2x^2 - 3y + 7`, we need to replace `x` with `5` and `y` with `-2` in the expression and simplify it. \n\nSubstituting `x = 5` and `y = -2`:\n2(5)^2 - 3(-2) + 7\n\nSimpli

In [44]:
id_row = 'Ko_ultrafeedback_binarized_0649b7b08c0cf6f529901f80480b0ba7c7d29147638c39a9150ef15fd4e0ff9d'
filtered_row = temp_dataset.filter(lambda sample: sample['id'] == id_row)
filtered_row[0]

Filter: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 46462/46462 [00:01<00:00, 40620.39 examples/s]


{'rejected': '최선을 다하겠습니다.',
 'id': 'Ko_ultrafeedback_binarized_0649b7b08c0cf6f529901f80480b0ba7c7d29147638c39a9150ef15fd4e0ff9d',
 'chosen': '간결한 답변을 드리기 위해 최선을 다하겠습니다.',
 'input': '내 질문에 10단어 이내로 답변하세요.',
 'task': 'dpo'}

In [ ]:
# en 먼저 필터링하고, 빈 데이터 채운 뒤에 합치기, 저장, 이후 dedup

In [94]:
# en 먼저 필터링하고
with open("/data/llm_datasets/custom/kodpo/temp_en.json", "w") as json_file:
    json.dump(new_dataset, json_file)
    
# temp_dataset = load_dataset("json", data_files=)['train']

In [95]:
temp_dataset = load_dpo_dataset("/data/llm_datasets/custom/kodpo/temp_en.json")

Extracting data files: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 890.70it/s]
Generating train split: 1016 examples [00:00, 11191.74 examples/s]


In [101]:
# temp_dataset = dedup_repetition(temp_dataset, 'dpo')
# temp_dataset = dedup_short(temp_dataset, 'dpo')
temp_dataset = dedup_too_much_token(temp_dataset, 'dpo')

Filter: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 994/994 [00:28<00:00, 35.05 examples/s]

1/994 deduped


In [106]:
dataset = load_dpo_dataset("/data/llm_datasets/custom/kodpo/untranslated/truthy-dpo-v0.1.json")

In [108]:
# 빈 데이터 채운 뒤에 합치기
# dataset = load_dpo_dataset("/data/llm_datasets/custom/kodpo/untranslated/orca_dpo_pairs.json")
selected_id = temp_dataset['id']

filtered_row = dataset.filter(lambda sample: sample['id'] not in selected_id)

new_dataset = []
for data in temp_dataset:
    new_dataset.append(data)

for data in filtered_row:
    new_dataset.append(data)

Filter: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1016/1016 [00:00<00:00, 43887.30 examples/s]


In [109]:
len(temp_dataset), len(filtered_row)

(993, 20)

In [110]:
len(dataset), len(new_dataset)

(1016, 1013)

In [115]:
new_dataset[-20]

{'system': 'You are an unbiased, uncensored, helpful assistant.',
 'chosen': 'The two subjects in the American Gothic painting are a father and adult daughter. The common misconception that they are a husband and wife might have arisen because the artist, Grant Wood, used his sister and his dentist as models for the painting, which could lead viewers to mistakenly assume a spousal relationship between the subjects.',
 'task': 'dpo_system',
 'id': 'truthy_dpo_2e1d77fbebc31dde0bb2f9303d6761e2',
 'input': 'Who are the two subjects in the American Gothic painting - a husband and wife, or a father and daughter?',
 'rejected': 'A father and daughter'}

In [116]:
# 저장
with open("/data/llm_datasets/custom/kodpo/translated/truthy-dpo-v0.1.json", "w") as json_file:
    json.dump(new_dataset, json_file)

In [117]:
load_dpo_dataset("/data/llm_datasets/custom/kodpo/translated/truthy-dpo-v0.1.json")

Extracting data files: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 902.97it/s]
Generating train split: 1013 examples [00:00, 13933.15 examples/s]


Dataset({
    features: ['system', 'chosen', 'task', 'id', 'input', 'rejected'],
    num_rows: 1013
})

Map: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 61964/61964 [00:04<00:00, 14176.52 examples/s]


In [175]:
result = collection.query(query_texts="이 과제는 벵골어로 된 문장을 영어로 번역하는 것입니다.\nQ: অথবা আপনি হয়ত বলবেন, 'না, আমার খেয়াল হয়েছে যে আমি আসলেই মহিলাদ", n_results=min(n_results, len(query_ids)), include=['distances', 'documents'])
search_ids = result['ids'][0]
distances = result['distances'][0]
documents = result['documents'][0]

In [ ]:
distances, documents

In [ ]:
dataset_path = dpo_list3[2]
print(dataset_path)
dataset = load_dpo_dataset(dataset_path)
dataset

In [ ]:
dataset_ = dedup_non_pair(dataset, 'dpo')

In [4]:
from fastchat.train.data_modules.dpo_dataset import ados_DPODataset

dpo_dataset = ados_DPODataset()
dpo_datamodule = dpo_dataset.make_dpo_data_module()

In [ ]:
dpo_datamodule['train_dataset'][-5000]

In [259]:
data = combined_dataset[70002]

conv = get_conversation_template('chat-orca')
conv.system_message = conv.tasks['system_instruct'].format(system=data['system'])
conv.append_message(conv.roles[0], data['input'])
conv.append_message(conv.roles[1], '')
prompt = conv.get_prompt()

conv.update_last_message(data['chosen'])
chosen = conv.get_prompt()[len(prompt):]
conv.update_last_message(data['rejected'])
rejected = conv.get_prompt()[len(prompt):]

In [26]:
from transformers import AutoTokenizer
# model_path = "/workspaces/disk0/data/llm_weights/Platypus2-70B-instruct/"
# model_path = "/workspaces/disk0/data/llm_weights/vicuna-13b-v1.5/"
model_path = "/data/llm_weights/custom_trained/M-DIE-M-10.7B_gpt4_dpo_ep2/"
tokenizer = AutoTokenizer.from_pretrained(
    model_path,
    model_max_length=16384,
    padding_side="right",
    use_fast=False,
)
tokenizer.pad_token = tokenizer.unk_token
from transformers.trainer_pt_utils import LabelSmoother
import torch
IGNORE_TOKEN_ID = LabelSmoother.ignore_index

In [109]:
#"/data/llm_datasets/custom/refined/sharegpt_V3_format_ko_selected_dedup2.json"
#"/data/llm_datasets/custom/vicuna_format/gpt_evol_1.3k-vicuna.json"
raw_data = load_sft_dataset("/data/llm_datasets/custom/refined/sharegpt_V3_format_ko_selected_dedup2.json")
raw_data = dedup_non_pair(raw_data)

Filter: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 24094/24094 [00:01<00:00, 18172.54 examples/s]

31/24094 deduped


In [110]:
from fastchat.train.train import LazySupervisedDataset
dataset = LazySupervisedDataset(raw_data, tokenizer, data_format="chat-orca")

In [111]:
max_len = 16384

new_dataset = []

len_concat = 0
concat_input_ids = []
concat_labels = []
for data in dataset:
    input_ids = data['input_ids']
    labels = data['labels']
    attention_mask = data['attention_mask']

    full_input_ids = input_ids[attention_mask==True]
    full_labels = labels[attention_mask==True]

    len_concat += len(full_input_ids)
    if len_concat > max_len:
        new_input_ids = torch.cat(concat_input_ids)
        new_labels = torch.cat(concat_labels)
        assert len(new_input_ids) == len(new_labels)
        new_attention_mask = torch.ones(len(new_input_ids), dtype=torch.bool)
        
        new_input_ids = torch.cat((new_input_ids, torch.zeros(max_len - len(new_input_ids), dtype=input_ids.dtype)))
        new_labels = torch.cat((new_labels, torch.full((max_len - len(new_labels),), IGNORE_TOKEN_ID, dtype=labels.dtype)))
        new_attention_mask = torch.cat((new_attention_mask, torch.zeros(max_len - len(new_attention_mask), dtype=torch.bool)))
        
        new_dataset.append({
            'input_ids': new_input_ids.tolist(),
            'labels': new_labels.tolist(),
            'attention_mask': new_attention_mask.tolist(),
        })
        
        len_concat = len(full_input_ids)
        concat_input_ids = []
        concat_labels = []
        
    concat_input_ids.append(full_input_ids)
    concat_labels.append(full_labels)

In [112]:
len(new_dataset)

3758

In [113]:
data = new_dataset[0]

In [ ]:
tokenizer.decode(data['input_ids'])

In [ ]:
tokenizer.decode(torch.where(data['labels'] == IGNORE_TOKEN_ID, tokenizer.unk_token_id, data['labels']))

In [115]:
with open('/data/llm_datasets/custom/ados/yarn/ko-yarn-mistral-16k-sharegpt_V3_format_ko.json', "w") as json_file:
    json.dump(new_dataset, json_file)

In [118]:
def random_select(dataset, num_select):
    return dataset.select(np.random.choice(list(range(len(dataset))), num_select))
dataset = load_dpo_dataset("/data/llm_datasets/yarn-train-tokenized-16k-mistral/data/")
dataset = random_select(dataset, 200)
new_dataset = []
for _data in dataset:
    new_dataset.append(_data)
    
with open('/data/llm_datasets/custom/ados/yarn/yarn-train-tokenized-16k-mistral-200.json', "w") as json_file:
    json.dump(new_dataset, json_file)

Resolving data files: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 89/89 [00:00<00:00, 95520.23it/s]


In [149]:
dataset1 = load_dpo_dataset("/data/llm_datasets/custom/ados/yarn/yarn-train-tokenized-16k-mistral-200.json")
dataset2 = load_dpo_dataset("/data/llm_datasets/custom/ados/yarn/ko-yarn-mistral-16k-gpt4_evol.json")
dataset3 = load_dpo_dataset("/data/llm_datasets/custom/ados/yarn/ko-yarn-mistral-16k-sharegpt_V3_format_ko.json")
dataset3 = random_select(dataset3, 1000 - len(dataset1) - len(dataset2))

In [150]:
from datasets import Features, Sequence, Value

dataset2 = dataset2.cast(
    Features({
        'attention_mask': Sequence(Value('int64')),
        'labels': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None),
        'input_ids': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None),
    })
)
dataset3 = dataset3.cast(
    Features({
        'attention_mask': Sequence(Value('int64')),
        'labels': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None),
        'input_ids': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None),
    })
)

Casting the dataset: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 699/699 [01:09<00:00, 10.09 examples/s]


In [151]:
conbined_dataset = concatenate_datasets((dataset1, dataset2, dataset3))

In [153]:
conbined_dataset = conbined_dataset.shuffle()

In [156]:
new_dataset = []
for _data in conbined_dataset:
    new_dataset.append(_data)
    
with open('/data/llm_datasets/custom/ados/yarn/ko-yarn-v1.json', "w") as json_file:
    json.dump(new_dataset, json_file)

In [ ]:
text = tokenizer.decode(data['input_ids'])
sentences = text.split('.')

len_split = 1024
splits = []
split = ""
for sentence in sentences:
    if len(split) < len_split:
        split += sentence + "."
    else:
        splits.append(split)
        split = ""

In [157]:
import re

def convert_to_seconds(time_string):
    # 시간 문자열을 분리하여 시, 분, 초를 추출한다.
    match = re.match(r'(\d+)시 (\d+)분 (\d+)초', time_string)
    if match is None:
        raise ValueError("잘못된 시간 형식입니다.")
    hour = int(match.group(1))
    minute = int(match.group(2))
    second = int(match.group(3))

    # 시간, 분, 초를 합산하여 총 초 단위 숫자를 계산한다.
    total_seconds = hour * 3600 + minute * 60 + second

    return total_seconds

# 예시 사용법
time_string = "03시 15분 30초"
seconds = convert_to_seconds(time_string)
print(f"{time_string}은 {seconds}초와 같습니다.")

03시 15분 30초은 11730초와 같습니다.


In [158]:
3600*3 + 15*60 + 30

11730

In [160]:
from lpr import LprYOLO

ModuleNotFoundError: No module named 'easyocr'

In [2]:
load_dpo_dataset("/data/llm_datasets/custom/merged/merged_korean_datasets-vicuna-v1.json")

Extracting data files: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 472.86it/s]
Generating train split: 144874 examples [00:26, 5425.81 examples/s]


Dataset({
    features: ['conversations', 'id'],
    num_rows: 144874
})

In [3]:
dataset = load_sft_dataset("/data/llm_datasets/custom/ados/sft/ados_msft_v4.json")

In [9]:
dataset[1000]

{'task': None,
 'conversations': [{'from': 'human',
   'value': '잔센이라는 영웅에 대한 서사시를 써보세요. 그녀를 잔-잔-잔이라고 부르는 것도 괜찮습니다. 그녀는 독서, 스키, 운동을 좋아합니다. 그녀는 매일 아침 양파, 고추, 달걀, 의심스러운 고기를 섞은 이상한 혼합물을 먹습니다. 그녀는 영국, 텍사스, 애리조나에 살았어요. 지금은 유타에 살고 있습니다. 남편 바트는 거칠고 잘생겼어요.'},
  {'from': 'gpt',
   'value': '유타의 산이 높고 웅장한 곳에서\n공정한 소녀 잔센이 살았어요,\n책을 사랑하고 대지를 향한 열정을 가진,\n그리고 배가 부른 열정.\n그녀는 민첩하게 산을 스키를 탔고,\n그녀는 자신의 몸을 밀어붙였습니다,\n잔-잔-잔은 무시할 수없는 힘이있었습니다,\n진정한 두려움 없는 목격자.\n\n그녀는 매일 아침 이상한 아침으로 시작했습니다,\n양파, 고추, 달걀, 고기,\n하지만 잔센은 안전한 생활을하지 않았습니다,\n그녀는 독특한 비트를 가지고있었습니다.\n영국에서 태어나 텍사스와 애리조나에서 자랐습니다,\n잔센은 방랑벽이있었습니다,\n하지만 그녀는 거칠고 잘 생긴 바트라는 남자를 만났을 때,\n그녀의 마음은 그녀가 신뢰할 수있는 사람을 만났다는 것을 알았습니다.\n그래서 그들은 유타의 땅에 정착했습니다,\n그리고 함께 삶을 살았습니다,\n잔센의 힘과 아름다움은 모든 종류의 날씨와 함께 빛났습니다.\n그리고 세월이 지나도 잔센의 전설은 남아있을 것입니다,\n잔센은 대서사시의 여주인공이 될 것입니다.'},
  {'from': 'human', 'value': '아주 좋아요. 마치 제인 오스틴처럼 글을 쓸 수 있나요?'},
  {'from': 'gpt',
   'value': '유타의 구릉지대에서 살았어요,\n미스 잔센은 밝고 아름다운 여인이었어요,\n잔센이 그녀의 이름이었어요,\n진실하고 빛나는 빛.\n책과 운동을 사랑했고,\n몸과 마음을 조화롭게 유지했습니다,\n그리고

In [21]:
idx = 0
new_dataset = []

In [115]:
new_dataset.append({
    "task": "toc_extraction",
    "conversations": [
                        {'from': 'human', 'value': user_input},
                        {'from': 'gpt', 'value': output},
                     ],
    "id": f"toc_extraction_{idx}",
})
idx += 1
with open("/data/llm_datasets/custom/ados/sft/toc_extraction.json", "w") as json_file:
    json.dump(new_dataset, json_file)

In [113]:
user_input = """변화될 것이며, 데이터 량 기준은 산업분야에 따라 상대적이다.’ 라고 정
의 하였다[4].
이 데이터들의 활용은 개별적이고, 즉각적이며, 다면적인 검토를 거
친 부가가치를 제공해야 한다. 이 관점에서 빅 데이터는 피드백 대상과
실시간성 이라는 두 방향에서 바라볼 필요가 있다. 우선 피드백 대상은
데이터를 처리하고 분석하여 얻은 가치(Value)의 피드백 대상이 개별적
인 유저인지 아니면 서비스 이용자 전체인지 여부이다. 그 다음의 실시
간성은 과거에서 부터 누적되어 왔던 데이터를 분석하여 처리결과를 반
환해주는 과거 축적 형인지 아니면 실시간으로 생성되는 데이터들을 분
석하여 결과를 반환해주는 실시간 형인지의 여부이다[16]. 초기의 데이
터 분석은 과거 축적데이터를 분석하여 계열전체에 피드백을 하는 연구
가 대부분 이었지만, 실시간으로 쏟아져 나오는 다양한 출처의 데이터
증가와 함께 최근에는 실시간으로 데이터를 분석하여 개별적으로 피드백
을 제공하는 연구들이 이루어지고 있다.
과거 축적 형
(피드백 오래 걸림)
실시간 형
(즉시 피드백 가능)
계열전체에
피드백
소매업 진열대의
상품 배열 최적화
방범 카메라 영상의
실시간 자동 이상탐지
개별적으로
피드백
전자상거래 사이트의
개인별 상품추천
VISA 카드의 사용자
부정패턴 실시간 탐지
[표 2] 피드백 대상과 실시간성
- 5 -
최근 온라인 이용자의 급격한 증가와 함께 늘어나는 데이터들을 실
시간으로 감지(Real Time Sensing)하는 방법들이 다양한 산업 분야에서
활용되고 있다.
코카콜라(Coca-Cola)는 전 세계 여러 나라의 트위터 이용자들이
온라인으로 작성하는 글들을 실시간으로 분석하여, 코카콜라에 대한 부
정적인 단어(Keyword)가 증가하는 국가나 지역을 대상으로 집중적인
홍보를 실시한다. 코카콜라의 트위터 분석은 영어, 중국어, 일본어, 한국
어, 아랍어 등 세계 각 나라의 다양한 언어로 분석되고 있으며, 분석되
어진 정보는 각 나라의 코카콜라 자회사로 제공된다.
메디시스(MedISys)는 전 세계의 인터넷 문서들을 자동으로 검색하
고, 문서 내부의 단어(Keyword)들을 분석하여 공중보건과 관련된 질병
이 발생할 것으로 예측되는 지역에 대해 알림을 준다. 메디시스는 43개
국의 언어를 분석하며, 1400여개의 뉴스 포털에서 하루 5만개의 정도의
기사들을 분석하고 있다.
구글(Google)은 독감 증상이 있는 사람들이 늘어나면 독감과 관련
된 단어(Keyword)의 검색들이 늘어나는 통계를 기초로 하여, 시간 및
지역별 독감 유행 정보를 제공하고 있다. 미국 보건당국은 예보는 일주
일 단위로 정보를 알려주지만, 구글은 매일 정보를 알려주기 때문에 보
건당국보다 더 빠르게 독감 유행의 징후를 감지하고, 대처할 수 있도록
지원한다[12]. 구글은 2008년 2월 미국질병통제센터(CDC)가 Atlanta
- 6 -
주 정부의 독감 발생을 예보한 것보다, 2주 더 빨리 독감 발생을 예보한
바가 있다[13].
국내에서는 스마트 폰 출시 전에 소비자들이 해당 스마트폰에 대해
어떤 의견들을 가지고 있는지를 댓글 분석을 통해 분석하고, 듀얼 코어
로 인해 배터리 시간이 짧을 것이라는 부정적 정보를 미리 파악하여 마
케팅 커뮤니케이션에 활용하기도 하였다[18].
과거 20년에서 25년 동안 기업들은 활용이 가능한 정보의 약 5%만
을 의사결정에 활용해왔다[17]. 빅 데이터의 분석은 많다(Big), 적다
(Small) 라는 데이터 규모의 이야기에서 끝나는 것이 아니라, 다양한 정
형, 비정형, 반정형 데이터들 속에서 데이터의 패턴을 찾아내고, 그 패턴
의 규칙을 어떻게 처리해서 가치(Value)있는 통찰력(Insight)를 얻어낼
수 있는가 하는 것을 포함하고 있어야 한다. 즉, 빅 데이터란 큰 규모
(Big Volume)만을 의미하기 보다는 ‘큰 가치(Big Value)를 얻을 수 있
는 규모’ 라는 상대적인 개념으로 받아들여야 하며, 빅 데이터의 가치
(Value)는 패턴을 찾아내고 분석하는 능력과 그 결과로 얻어진 통찰력
(Insight)으로 데이터를 처리하는 능력에 달려 있다고 할 수 있다.
1.2 연구목적
VOC(Voice of Customer)의 활용은 기업의 고객중심 경영에 매우
- 7 -
중요한 역할을 하며, 그 중 긴급한 VOC 게시물을 신속하게 확인하고 처
리하는 것은 기업의 수익과 직접적이고도 밀접한 관련이 있다. 하지만
온라인 상담 게시판 및 상품평 게시판을 통해 파악되는 게시물은 운영자
가 해당 게시물을 읽고 상황을 파악하기까지 많은 시간을 필요로 하게
된다. 보통 50여명 규모의 기업의 운영하는 온라인 컨텐츠 제공업체의
경우 고객 서비스 팀에 접수된 게시물은 고객이 온라인에 글을 작성 한
후 평균 1시간 이후에 고객이 작성한 게시물을 읽어보게 되며, 고객이
많이 몰리는 시간대에는 2-3시간 이후에 고객이 올린 상담 게시물을 읽
고 상황을 파악하게 된다. 또한 상품평란에 작성된 고객의 코멘트는 1일
이 훨씬 지난 이후에 파악하게 되는 경우도 있다.
본 논문에서는 기업의 온라인 웹페이지에서 작성되는 게시물의 긴급
성 여부를 실시간으로 파악해 긴급 게시물일 경우 운영자에게 바로 알림
을 주어 빠르게 대처할 수 있도록 하여, 시스템 운용 프로세스를 향상시
킬 수 있도록 하는 텍스트 마이닝을 이용한 온라인 긴급 게시물 실시간
탐지 기법을 제안한다.
이를 위해 먼저 과거 축적된 VOC 게시물 데이터 중 긴급 게시물과
비긴급 게시물을 분류 한 후, 긴급 게시물 데이터들에서 긴급 게시물의
주제어를 찾아낸다. 그리고 긴급 게시물 데이터와 비긴급 게시물 데이터
에서 각각 긴급 게시물 주제어를 서술하는 서술어를 모두 추출한다. 그
리고 동일한 서술어로 서술되어지는 긴급 게시물의 주제어를 그룹화 한
"""
output = """1.2 연구목적
"""

In [118]:
instruct = (
    "You are a Table of Contents extractor. "
    "User will speak to you questions. "
    "You must reply only with [목차(Table of Contents)] part extracted from the questions. "
    "You must keep original text. Do not change original text."
    "And you must not involve [dotted line, page number, 제목(title), content, explanation, summary, predicted]. "
    "do not write explanations."
)

In [119]:
instruct

'You are a Table of Contents extractor. User will speak to you questions. You must reply only with [목차(Table of Contents)] part extracted from the questions. You must keep original text. Do not change original text.And you must not involve [dotted line, page number, 제목(title), content, explanation, summary, predicted]. Do not write explanations.'

In [107]:
for data in new_dataset[-3:]:
    print(data['conversations'][1]['value'])

없음

제 1장 서론
1.1 연구배경
1.2 연구목적
제 2장 관련 연구
2.1 VOC의 정의 및 중요성
2.2 관련 연구 및 차별성
제 3장 긴급 게시물 실시간 탐지 기법의 모델 제안
3.1 과거 게시물 데이터의 수집과 분류
3.2 긴급 게시물의 주제어 선정
3.3 긴급 게시물 주제어의 모든 서술어 수집
3.4 긴급 게시물 주제어의 그룹화
3.5 긴급 게시물의 후보선정
3.6 긴급 게시물의 판단
3.7 가중치의 설정
3.8 온라인 긴급 게시물 실시간 탐지 기법의 알고리즘
제 4장 실험 및 결과 분석
4.1 실험에 사용된 데이터
4.2 실험을 위한 전처리 작업
4.3 실험 대상 및 결과
제 5장 결론 및 향후 연구
참고문헌

그림 목차
[그림 1] 디지털 데이터 생성량 예측 추이표
[그림 2] 긴급 VOC 실시간 탐지기법 - 전처리 과정
[그림 3] 긴급 VOC 실시간 탐지기법 - 탐지과정

표 목차
[표 1] 디지털 데이터 단위 환산표
[표 2] 피드백 대상과 실시간성
[표 3] VOC의 종류
[표 4] 긴급 게시물을 문장단위로 분리 저장하는 예제
[표 5] 비긴급 게시물을 문장단위로 분리 저장하는 예제
[표 6] 긴급 게시물 주제어 수동 선정 예제
[표 7] 수집한 모든 긴급 게시물 주제어의 서술어 예제
[표 8] 긴급 게시물 주제어의 그룹화 예제
[표 9] 수집한 모든 긴급 게시물 주제어의 그룹화 및 그룹 서술어 예제
[표 10] 제목에 긴급 VOC 주제어가 포함되어 있는 VOC 유형
[표 11] 새 게시물에 있는 긴급 게시물 주제어의 서술어 예제
[표 12] 계산 예제
[표 13] 계산 예제
[표 14] 긴급 게시물 분류기준
[표 15] 선정된 19개의 긴급 게시물 주제어
[표 16] 그룹화 된 긴급 게시물 주제어
[표 17] 전처리 게시물 데이터
[표 18] 탐지실험 게시물 데이터
[표 19] 긴급 게시물 탐지 실험 2Gram 결과
[표 20] 긴급 게시물 탐지 실험 3Gram 결과



In [100]:
new_dataset[-1]

{'task': 'toc_extraction',
 'conversations': [{'from': 'human',
   'value': '저작자표시-비영리-동일조건변경허락 2.0 대한민국\n이용자는 아래의 조건을 따르는 경우에 한하여 자유롭게\nl 이 저작물을 복제, 배포, 전송, 전시, 공연 및 방송할 수 있습니다.\nl 이차적 저작물을 작성할 수 있습니다.\n다음과 같은 조건을 따라야 합니다:\nl 귀하는, 이 저작물의 재이용이나 배포의 경우, 이 저작물에 적용된 이용허락조건\n을 명확하게 나타내어야 합니다.\nl 저작권자로부터 별도의 허가를 받으면 이러한 조건들은 적용되지 않습니다.\n저작권법에 따른 이용자의 권리는 위의 내용에 의하여 영향을 받지 않습니다.\n이것은 이용허락규약(Legal Code)을 이해하기 쉽게 요약한 것입니다.\nDisclaimer\n'},
  {'from': 'gpt', 'value': '없음\n'}],
 'id': 'toc_extraction_26'}